# Pareto Conditioned networks

Using the modified dam environment (less variance in the inflow), test PCN. This modifications is necessary because PCN expects deterministic transitions.

Also a modification to the original PCN code in morl-baselines is necessary in order to support the continuous actions of the environment:
- continuous_actions is a parameter given to the algorithm
- During training, actions are no longer sampled from a categorical distribution as in the [original paper](https://www.ifaamas.org/Proceedings/aamas2022/pdfs/p1110.pdf). This might hinder exploration...
- The logsoftmax function of the original model is no longer used, instead its output is a single value corresponding to an action
- The loss used in the model is now MSE instead of CE (regression loss vs classification)

In [2]:
import mo_gymnasium as mo_gym
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from morl_baselines.multi_policy.pcn.pcn import PCN

"""
Helper function to plot pf
"""
# TODO: add no objectives as parameter (later)
def plot_pf(file):
    columns = ["objective_1", "objective_2"]
    df = pd.read_csv(file, usecols=columns)

    plt.plot(df.objective_1, df.objective_2, 'o')
    plt.xlabel('Cost due to excess water level wrt flooding threshold upstream')
    plt.ylabel('Deficit in water supply wrt demand')
    plt.show()

## First attempt

- Dam inflow: mean = 40.0 and stdev = 1.0 
- Actions capped between [0.0;250.0]
- Scaling factor for desired horizon/return = [0.1, 0.1]
- Hypervolume ref point = [-100, -1000]

In [3]:
env = mo_gym.make('water-reservoir-v0', normalized_action=False, nO=2, penalize=True, time_limit=100)

PCNAgent = PCN(env, np.array((0.1, 0.1, 0.1), dtype=np.float32), continuous_actions=True)

max_return = np.zeros(2)

desired_return = np.array((-20.0, -630.0))
PCNAgent.set_desired_return_and_horizon(desired_return=desired_return, desired_horizon=0)

PCNAgent.train(10000000, env, np.array((-100.0,-1000), dtype=np.float32), num_step_episodes=100,max_return=max_return)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: liam-mertens02 (vub-ai). Use `wandb login --relogin` to force relogin


wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in /tmp/PCN
c:\Users\liamm\anaconda3\lib\site-packages\morl_baselines\multi_policy\pcn\pcn.py:190: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  th.tensor(obs).to(self.device),
c:\Users\liamm\anaconda3\lib\site-packages\morl_baselines\multi_policy\pcn\pcn.py:404: RuntimeWarning: overflow encountered in multiply
  ent = np.sum(-np.exp(lp) * lp)
c:\Users\liamm\anaconda3\lib\site-packages\morl_baselines\multi_policy\pcn\pcn.py:404: RuntimeWarning: overflow encountered in exp
  ent = np.sum(-np.exp(lp) * lp)


step 60000 	 return [-7082.882  -8074.1567], ([ 6.5656724 22.188858 ]) 	 loss 8.543E+03
step 70000 	 return [-6626.084 -7616.352], ([ 2.439899 23.981667]) 	 loss 4.269E+03
step 80000 	 return [-6622.7817 -7617.29  ], ([ 2.0073726 21.380568 ]) 	 loss 3.058E+03
step 90000 	 return [-6606.732 -7597.325], ([ 2.0039642 20.370197 ]) 	 loss 2.254E+03
step 100000 	 return [-6607.9556 -7603.368 ], ([ 1.8573393 20.589802 ]) 	 loss 1.112E+03


wandb: WARNING Step cannot be set when using syncing with tensorboard. Please log your step values as a metric such as 'global_step'


step 110000 	 return [-6606.01  -7599.614], ([ 1.8485326 21.356756 ]) 	 loss 9.544E+01
step 120000 	 return [-6608.456  -7602.7637], ([ 1.7181865 22.72263  ]) 	 loss 3.081E+01
step 130000 	 return [-6609.399  -7603.0107], ([ 1.9992337 21.411774 ]) 	 loss 2.368E+01
step 140000 	 return [-6606.9736 -7597.839 ], ([ 1.9234885 22.724628 ]) 	 loss 2.476E+01
step 150000 	 return [-6605.486  -7601.4287], ([ 1.6822094 24.264341 ]) 	 loss 1.563E+01
step 160000 	 return [-6601.6533 -7595.4688], ([ 1.851816 23.232746]) 	 loss 2.511E+01
step 170000 	 return [-6599.0127 -7590.5093], ([ 1.6485417 21.793571 ]) 	 loss 1.866E+01
step 180000 	 return [-6598.538  -7589.8276], ([ 1.5868927 21.772291 ]) 	 loss 1.480E+01
step 190000 	 return [-6599.2964 -7590.256 ], ([ 1.6704288 22.318876 ]) 	 loss 1.119E+01
step 200000 	 return [-6597.275  -7589.1504], ([ 1.6236256 21.443516 ]) 	 loss 1.150E+01
step 210000 	 return [-6597.937  -7588.1626], ([ 2.4227304 21.657925 ]) 	 loss 1.217E+01
step 220000 	 return [-66

KeyboardInterrupt: 

Stopped training at 30% mark because no improvement is seen...